# FinGPT Sentiment Analysis & Price Forecasting

This notebook uses:
1. **FinGPT Sentiment** - For sentiment classification of financial news
2. **FinGPT Forecaster** - For predicting actual price changes

Both models run locally on your Mac (M-series with MPS acceleration).

**Memory Optimized Version:** Models are loaded one at a time and unloaded after use to fit within 24GB RAM.

In [1]:
# pyspark packages
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType

#other needed packages
import re
import os

# Set JAVA_HOME for PySpark
os.environ['JAVA_HOME'] = '/opt/homebrew/opt/openjdk@17'

spark = SparkSession.builder \
    .appName("stock market preds") \
    .config("spark.driver.host", "127.0.0.1") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/18 03:29:43 WARN Utils: Your hostname, Jeffreys-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 10.0.0.17 instead (on interface en0)
26/01/18 03:29:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/18 03:29:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/18 03:29:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/01/18 03:29:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
26/01/18 03:29:44 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
# functions to import data, run SQL from file and save back to file
def import_csv_to_table(table_name, file, format_cols):
    df = spark.read.csv(file, header=True, quote="\"",
                        escape="\"", multiLine=True, inferSchema=True)
    if format_cols:
        cols_formatted = [re.sub(r"[^a-zA-Z0-9\s]", "", col_name).lower().replace(" ", "_") for col_name in df.columns]
        df = df.toDF(*cols_formatted)
    df.createOrReplaceTempView(f"{table_name}")
    return df

def sql_step(file):
    with open(file, 'r', encoding='utf-8') as f:
        sql_text = f.read()
    return spark.sql(sql_text)

In [3]:
news = import_csv_to_table("news", "raw_data/news_data.csv", False)
stocks = import_csv_to_table("stocks", "raw_data/stock_data.csv", False)

In [ ]:
feature_set = sql_step("sql/sentiment_data_prep_fingpt.sql")
feature_set.show(10, truncate=False)

+---------------+------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------+
|news_article_id|symbol|news_date |article_text                                                                                                                                                                                                                                                                                                                                                            |percent_daily_price_change|
+---------------+------+----------+---------------------------------------------------------------------------------------------------------------------

In [5]:
# Setup device (MPS for M-series Mac, CUDA for GPU, CPU as fallback)
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")

Using device: mps
PyTorch version: 2.9.1


In [13]:
# FinGPT Sentiment Scoring Function (model loaded on-demand)
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import pandas as pd
from tqdm import tqdm
import gc
import warnings
warnings.filterwarnings('ignore')

# Updated to use available FinGPT models on Hugging Face
# Using Llama2 7B version from oliverwang15 (the FinGPT author)
BASE_MODEL = "NousResearch/Llama-2-7b-hf"
SENTIMENT_ADAPTER = "oliverwang15/FinGPT_v32_Llama2_Sentiment_Instruction_LoRA_FT"

def load_fingpt_sentiment():
    """Load FinGPT Sentiment model with CPU offloading for memory efficiency."""
    print("Loading FinGPT Sentiment model...")
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
    tokenizer.pad_token = tokenizer.eos_token
    
    # Use CPU for loading, then move to MPS for inference
    # This prevents memory fragmentation issues
    model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        torch_dtype=torch.float16,
        device_map="mps",  # Direct to MPS instead of auto
        low_cpu_mem_usage=True,
        offload_folder="offload"  # Offload to disk if needed
    )
    model = PeftModel.from_pretrained(model, SENTIMENT_ADAPTER)
    model.eval()
    print("FinGPT Sentiment model loaded successfully!")
    return tokenizer, model

def unload_model(model, tokenizer):
    """Unload model and free memory."""
    del model
    del tokenizer
    gc.collect()
    if torch.backends.mps.is_available():
        torch.mps.empty_cache()
    elif torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("Model unloaded and memory freed.")

def score_sentiment_fingpt(article_text, tokenizer, model, max_length=512):
    """Score sentiment using FinGPT model."""
    prompt = f"""Instruction: What is the sentiment of this news? Please choose an answer from {{negative/neutral/positive}}.
Input: {article_text[:500]}
Answer: """
    
    try:
        inputs = tokenizer(prompt, return_tensors="pt", max_length=max_length, truncation=True)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=10,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )
        
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        answer = response.split("Answer:")[-1].strip().lower()
        
        if "positive" in answer:
            return "positive", 0.85
        elif "negative" in answer:
            return "negative", 0.85
        else:
            return "neutral", 0.7
    except Exception as e:
        print(f"Error: {e}")
        return "neutral", 0.5

print("Sentiment functions defined.")

Sentiment functions defined.


In [8]:
# Authenticate with Hugging Face using API key
from huggingface_hub import login
import json

with open("api_keys.json", "r") as f:
    api_keys = json.load(f)

login(token=api_keys["HUGGINGFACE_KEY"])
print("Successfully logged in to Hugging Face!")

Successfully logged in to Hugging Face!


In [14]:
# STEP 1: Score all articles with FinGPT Sentiment
# Model is loaded, used, then unloaded to free memory

df = feature_set.toPandas()
print(f"Total articles to score: {len(df)}")

# Load sentiment model
tokenizer_sentiment, model_sentiment = load_fingpt_sentiment()

# Test on a sample first
test_text = "Goldman Sachs upgrades Apple to Buy rating with price target of $200"
sentiment, conf = score_sentiment_fingpt(test_text, tokenizer_sentiment, model_sentiment)
print(f"Test: '{test_text}'")
print(f"Sentiment: {sentiment}, Confidence: {conf}\n")

# Score all articles
sentiments, sentiment_confs = [], []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Scoring Sentiment"):
    text = row['article_text']
    sent, sent_conf = score_sentiment_fingpt(text, tokenizer_sentiment, model_sentiment)
    sentiments.append(sent)
    sentiment_confs.append(sent_conf)

df['sentiment'] = sentiments
df['sentiment_confidence'] = sentiment_confs

print(f"\nSentiment distribution:\n{df['sentiment'].value_counts()}")

# Unload sentiment model to free memory for forecaster
unload_model(model_sentiment, tokenizer_sentiment)

Total articles to score: 35430
Loading FinGPT Sentiment model...


Loading checkpoint shards: 100%|██████████| 2/2 [01:23<00:00, 41.59s/it]
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


FinGPT Sentiment model loaded successfully!
Test: 'Goldman Sachs upgrades Apple to Buy rating with price target of $200'
Sentiment: neutral, Confidence: 0.7



Scoring Sentiment:   1%|          | 184/35430 [04:24<14:05:25,  1.44s/it]


KeyboardInterrupt: 

In [15]:
# Clear memory from previous failed attempts
import gc
import torch

# Force garbage collection
gc.collect()

# Clear MPS cache
if torch.backends.mps.is_available():
    torch.mps.empty_cache()
    
print("Memory cleared!")

Memory cleared!


In [ ]:
# FinGPT Forecaster Functions

def load_fingpt_forecaster():
    """Load FinGPT Forecaster model."""
    print("Loading FinGPT Forecaster model...")
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
    tokenizer.pad_token = tokenizer.eos_token
    
    model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        torch_dtype=torch.float16,
        device_map="auto",
        low_cpu_mem_usage=True
    )
    model = PeftModel.from_pretrained(model, FORECASTER_ADAPTER)
    model.eval()
    print("FinGPT Forecaster model loaded successfully!")
    return tokenizer, model

def predict_price_change_fingpt(article_text, symbol, tokenizer, model, max_length=512):
    """Predict price direction using FinGPT Forecaster."""
    prompt = f"""Instruction: Based on the following news, predict if the stock price will go up, down, or stay stable. Answer with: up/down/stable.
Company: {symbol}
News: {article_text[:400]}
Prediction: """
    
    try:
        inputs = tokenizer(prompt, return_tensors="pt", max_length=max_length, truncation=True)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=15,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )
        
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        answer = response.split("Prediction:")[-1].strip().lower()
        
        if "up" in answer:
            return "up", 0.7
        elif "down" in answer:
            return "down", 0.7
        else:
            return "stable", 0.6
    except Exception as e:
        print(f"Error: {e}")
        return "stable", 0.5

print("Forecaster functions defined.")

In [ ]:
# STEP 2: Predict price direction with FinGPT Forecaster
# Sentiment model was unloaded, now load forecaster

# Load forecaster model
tokenizer_forecast, model_forecast = load_fingpt_forecaster()

# Test on a sample first
test_text = "Apple reports record quarterly earnings beating analyst expectations"
direction, conf = predict_price_change_fingpt(test_text, "AAPL", tokenizer_forecast, model_forecast)
print(f"Test: '{test_text}'")
print(f"Predicted direction: {direction}, Confidence: {conf}\n")

# Predict for all articles
predictions, prediction_confs = [], []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Predicting Price Direction"):
    text = row['article_text']
    symbol = row['symbol']
    pred, pred_conf = predict_price_change_fingpt(text, symbol, tokenizer_forecast, model_forecast)
    predictions.append(pred)
    prediction_confs.append(pred_conf)

df['predicted_direction'] = predictions
df['prediction_confidence'] = prediction_confs

print(f"\nPrediction distribution:\n{df['predicted_direction'].value_counts()}")

# Unload forecaster model to free memory
unload_model(model_forecast, tokenizer_forecast)

In [ ]:
df.head(10)

In [ ]:
# Export results
os.makedirs("processed_data", exist_ok=True)
df.to_csv("processed_data/fingpt_news_classifications.csv", index=False)
print(f"Exported {len(df)} rows to processed_data/fingpt_news_classifications.csv")

In [ ]:
# Evaluate FinGPT Performance
import matplotlib.pyplot as plt
import numpy as np

# Calculate actual direction
df['actual_direction'] = df['percent_daily_price_change'].apply(
    lambda x: 'up' if x > 0.005 else ('down' if x < -0.005 else 'stable')
)

# Binary accuracy (up/down only, excluding stable)
df_binary = df[df['predicted_direction'] != 'stable'].copy()
df_binary['actual_binary'] = df_binary['percent_daily_price_change'].apply(lambda x: 'up' if x > 0 else 'down')
df_binary['correct'] = df_binary['predicted_direction'] == df_binary['actual_binary']

fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# 1. Sentiment Distribution
sent_counts = df['sentiment'].value_counts()
colors_sent = {'positive': 'green', 'neutral': 'gray', 'negative': 'red'}
axes[0,0].bar(sent_counts.index, sent_counts.values, color=[colors_sent[s] for s in sent_counts.index])
axes[0,0].set_title('FinGPT Sentiment Distribution')
axes[0,0].set_ylabel('Count')

# 2. Prediction Distribution
pred_counts = df['predicted_direction'].value_counts()
colors_pred = {'up': 'green', 'stable': 'gray', 'down': 'red'}
axes[0,1].bar(pred_counts.index, pred_counts.values, color=[colors_pred.get(p, 'blue') for p in pred_counts.index])
axes[0,1].set_title('FinGPT Price Prediction Distribution')
axes[0,1].set_ylabel('Count')

# 3. Actual vs Predicted
actual_counts = df['actual_direction'].value_counts()
x = np.arange(3)
width = 0.35
labels = ['up', 'stable', 'down']
pred_vals = [pred_counts.get(l, 0) for l in labels]
actual_vals = [actual_counts.get(l, 0) for l in labels]
axes[0,2].bar(x - width/2, pred_vals, width, label='Predicted', alpha=0.7)
axes[0,2].bar(x + width/2, actual_vals, width, label='Actual', alpha=0.7)
axes[0,2].set_xticks(x)
axes[0,2].set_xticklabels(labels)
axes[0,2].set_title('Predicted vs Actual Distribution')
axes[0,2].legend()

# 4. Price Change by Sentiment
means = df.groupby('sentiment')['percent_daily_price_change'].mean() * 100
axes[1,0].bar(means.index, means.values, color=[colors_sent[s] for s in means.index])
axes[1,0].set_title('Mean Price Change by Sentiment (%)')
axes[1,0].axhline(0, color='black', linestyle='--', alpha=0.5)

# 5. Prediction Accuracy by Direction
if len(df_binary) > 0:
    acc_by_pred = df_binary.groupby('predicted_direction')['correct'].mean() * 100
    axes[1,1].bar(acc_by_pred.index, acc_by_pred.values, color=[colors_pred.get(p, 'blue') for p in acc_by_pred.index])
    axes[1,1].axhline(50, color='red', linestyle='--', label='Random baseline')
    axes[1,1].set_title('Prediction Accuracy by Direction (%)')
    axes[1,1].legend()

# 6. Confusion Matrix
from sklearn.metrics import confusion_matrix
if len(df_binary) > 0:
    cm = confusion_matrix(df_binary['actual_binary'], df_binary['predicted_direction'], labels=['up', 'down'])
    im = axes[1,2].imshow(cm, cmap='Blues')
    axes[1,2].set_xticks([0, 1])
    axes[1,2].set_yticks([0, 1])
    axes[1,2].set_xticklabels(['Up', 'Down'])
    axes[1,2].set_yticklabels(['Up', 'Down'])
    axes[1,2].set_xlabel('Predicted')
    axes[1,2].set_ylabel('Actual')
    axes[1,2].set_title('Confusion Matrix')
    for i in range(2):
        for j in range(2):
            axes[1,2].text(j, i, cm[i, j], ha='center', va='center', 
                          color='white' if cm[i, j] > cm.max()/2 else 'black')

plt.tight_layout()
plt.show()

# Summary Statistics
print("="*70)
print("FINGPT PERFORMANCE SUMMARY")
print("="*70)
print(f"\nTotal articles analyzed: {len(df)}")
print(f"\nSentiment Distribution:")
for s in ['positive', 'neutral', 'negative']:
    count = len(df[df['sentiment'] == s])
    pct = count / len(df) * 100
    mean_change = df[df['sentiment'] == s]['percent_daily_price_change'].mean() * 100
    print(f"  {s:10}: {count:5} ({pct:5.1f}%) - Mean price change: {mean_change:+.3f}%")

print(f"\nPrediction Accuracy (up/down only):")
if len(df_binary) > 0:
    overall_acc = df_binary['correct'].mean() * 100
    print(f"  Overall: {overall_acc:.1f}%")
    for pred in ['up', 'down']:
        subset = df_binary[df_binary['predicted_direction'] == pred]
        if len(subset) > 0:
            acc = subset['correct'].mean() * 100
            print(f"  {pred:10}: {acc:.1f}% ({len(subset)} predictions)")

In [ ]:
# Compare Sentiment vs Forecaster accuracy
print("\n" + "="*70)
print("SENTIMENT vs FORECASTER COMPARISON")
print("="*70)

# Sentiment-based direction
df['sentiment_direction'] = df['sentiment'].map({'positive': 'up', 'negative': 'down', 'neutral': 'stable'})
df_sent_binary = df[df['sentiment'] != 'neutral'].copy()
df_sent_binary['actual_binary'] = df_sent_binary['percent_daily_price_change'].apply(lambda x: 'up' if x > 0 else 'down')
df_sent_binary['sent_correct'] = df_sent_binary['sentiment_direction'] == df_sent_binary['actual_binary']

print(f"\nSentiment-based accuracy (pos/neg → up/down): {df_sent_binary['sent_correct'].mean()*100:.1f}%")
if len(df_binary) > 0:
    print(f"Forecaster accuracy (up/down predictions):     {df_binary['correct'].mean()*100:.1f}%")

print("\nNote: If both are around 50%, news sentiment may not strongly predict next-day returns.")